### CMSC 170: Laboratory Exercise 3
##### The 8-Puzzle Problem
* Dalion, Adrian Lloyd
* Salcedo, Chris Samuel
* Suyman, Ann Junah

#### Task 1.A: Define the Problem

##### 1. What is the problemt that needs a solution?

The 8-puzzle problem is a sliding puzzle that consists of 8-numbered tiles, labeled 1-8, and an empty space in a 3x3 grid. The challenge is rearranging the tiles through that empty space to achieve a specific goal state.

##### 2. What is the initial state?

The initial state is any valid state/configuration of the problem, serving as the starting point to where the problem-solving begins.

##### 3. What is the goal state?

The goal state is the desired state/configuration that is to be achieved. The most common goal state is the numerically arranged state [[1,2,3],[4,5,6],[7,8,0]].

##### 4. What are the valid actions?
The valid actions consist of four moves, representing the moves of the tiles into the empty space.

* UP: Move the tile below the empty space upward.
* DOWN: Move the tile above the empty space doenward
* LEFT: Move the tile on the right of the empty space to the left
* RIGHT: Move the tile on the left of the empty space to the right

Of course, the validity of a move or an action relies on the current condition of the empty space. If the empty space is located in a side or corner of the grid then there will not be enough tiles arround it to fullfill the allowability of the four listed moves.

##### 5. Whatis/are the functions that validate whether a state is valid?

State validation check include constraints where,
1. The board should only contain exactly 8 numbered tiles and one empty space
2. All tiles are within the 3x3 boundaries
3. No duplicate numbers exist, and
4. The state is reachable from the initial state

#### References:
* https://www.geeksforgeeks.org/dsa/8-puzzle-problem-using-branch-and-bound/

#### Task 1.B: Define the Breadth-First Search

##### 1. What is the main idea behind the breadth-first search (BFS) algorithm, and how does it explore a graph or tree?

The Breadth-First Search (BFS) is a search algorithm that explores nodes by level, starting from the root note, visiting all nodes at depth `d` before exploring nodes `d+1`. In the current puzzle, the algorithm explores all possible moves from the current state before moving to states that require more moves. This way, the shortest possible path is guaranteed to be outputted.

##### 2. What data structure does BFS use, and why is it important? How does BFS differ from depth-first search (DFS)?

BFS uses a queue (FIFO) data structure to ensure that nodes are processed in the order they were discovered and therefore guaranteeing level-by-level exploration. On the other hand, DFS uses stack (LIFO) and explores more depth-wise (hence the name), which although may find a solution that is not the most optimized, it is better memory-wise. BFS is used mostly on problems such as bipartite graphs and shortest paths, while DFS shows its strengths on problems such as acyclic graphs and finding strongly connected components. 

#### 3. What is the time complexity of BFS for a graph with V vertices and E edges?

The time complexity of BFS is O(V+E) while its space complexity is O(V) for storing the queue and visited states.

#### References:
* https://www.geeksforgeeks.org/dsa/breadth-first-search-or-bfs-for-a-graph/
* https://www.geeksforgeeks.org/dsa/difference-between-bfs-and-dfs/


#### Task 2: 8-Puzzle Game in Python

In [16]:
from typing import List, Tuple, Optional
import copy
from IPython.display import clear_output

In [13]:
# core 8-puzzle problem definition
class EightPuzzle:
    # represents the 8-puzzle problem with state validation and action generation.
    
    def __init__(self, initial_state: Optional[List[List[int]]] = None, goal_state: Optional[List[List[int]]] = None):
        # initialize with default states if none provided
        self.initial_state = initial_state or [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
        self.goal_state = goal_state or [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
    
    def is_valid_state(self, state: List[List[int]]) -> bool:
        # check if state is a proper 3x3 configuration with numbers 0-8
        if not state or len(state) != 3:
            return False
        
        flat_state = []
        for row in state:
            if len(row) != 3:
                return False
            flat_state.extend(row)
        
        return sorted(flat_state) == list(range(9))
    
    def find_empty_position(self, state: List[List[int]]) -> Optional[Tuple[int, int]]:
        # find the position of the empty space (0)
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    return (i, j)
        return None
    
    def get_valid_actions(self, state: List[List[int]]) -> List[str]:
        # get all possible moves from the current state
        empty_pos = self.find_empty_position(state)
        if not empty_pos:
            return []
        
        row, col = empty_pos
        valid_actions = []
        
        # define possible moves and their direction offsets
        moves = {
            'up': (1, 0),
            'down': (-1, 0),
            'left': (0, 1),
            'right': (0, -1)
        }
        
        for action, (dr, dc) in moves.items():
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                valid_actions.append(action)
        
        return valid_actions
    
    def apply_action(self, state: List[List[int]], action: str) -> Optional[List[List[int]]]:
        # apply a move to the state and return the new state
        empty_pos = self.find_empty_position(state)
        if not empty_pos:
            return None
        
        row, col = empty_pos
        new_state = [row[:] for row in state]  # deep copy
        
        moves = {
            'up': (1, 0),
            'down': (-1, 0),
            'left': (0, 1),
            'right': (0, -1)
        }
        
        if action in moves:
            dr, dc = moves[action]
            new_row, new_col = row + dr, col + dc
            
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                # swap empty space with adjacent tile
                new_state[row][col] = new_state[new_row][new_col]
                new_state[new_row][new_col] = 0
                return new_state
        
        return None
    
    def is_goal_state(self, state: List[List[int]]) -> bool:
        # check if the current state matches the goal state
        return state == self.goal_state
    
    def is_solvable(self) -> bool:
        # check if the puzzle is solvable using inversion count
        def count_inversions(state: List[List[int]]) -> int:
            flat_state = [num for row in state for num in row if num != 0]
            inversions = 0
            for i in range(len(flat_state)):
                for j in range(i + 1, len(flat_state)):
                    if flat_state[i] > flat_state[j]:
                        inversions += 1
            return inversions
        
        initial_inversions = count_inversions(self.initial_state)
        goal_inversions = count_inversions(self.goal_state)
        
        # for 3x3 puzzle, solvability depends on inversion parity
        return (initial_inversions % 2) == (goal_inversions % 2)
    
    def display_state(self, state: List[List[int]], title: str = "state"):
        # display the state in a formatted way
        print(f"\n{title}:")
        print("┌─────┬─────┬─────┐")
        for i, row in enumerate(state):
            row_str = "│"
            for num in row:
                if num == 0:
                    row_str += "     │"
                else:
                    row_str += f"  {num}  │"
            print(row_str)
            if i < 2:
                print("├─────┼─────┼─────┤")
        print("└─────┴─────┴─────┘")

In [14]:
# interactive game extending the core problem
class EightPuzzleGame(EightPuzzle):
    
    def __init__(self):
        super().__init__()
        self.current_state = None
        self.moves_count = 0
    
    def display_instructions(self):
        # display game instructions
        print()
        print("8-Puzzle Game Start")
        print()
        print("\nThe 8-puzzle problem is a 3x3 board with 8 tiles numbered from 1 to 8")
        print("and one empty space (represented by 0).")
        print("\nThe objective is to begin with an arbitrary configuration of tiles,")
        print("and move them to place the numbered tiles to match the final configuration.")
        print("\nRules:")
        print("1. Input the initial state of the puzzle using this format:")
        print("   [1,2,3,4,0,8,5,6,7] (where 0 represents the empty space)")
        print("\n2. Use the following keys to move tiles into the empty space:")
        print("   'W' or 'w' → move tile up into empty space")
        print("   'S' or 's' → move tile down into empty space") 
        print("   'A' or 'a' → move tile left into empty space")
        print("   'D' or 'd' → move tile right into empty space")
        print("   'Q' or 'q' → quit the game")
        print()
    
    def setup_initial_state(self) -> bool:
        # get and validate initial state from user input
        while True:
            try:
                print("\nEnter the initial state of the puzzle:")
                user_input = input("Format: [1,2,3,4,0,8,5,6,7] → ")
                
                # parse input
                numbers = eval(user_input)
                if len(numbers) != 9:
                    raise ValueError("Must contain exactly 9 elements")
                
                # convert to 3x3 board
                board = [numbers[i:i+3] for i in range(0, 9, 3)]
                
                if self.is_valid_state(board):
                    self.initial_state = board
                    self.current_state = [row[:] for row in board]  # deep copy
                    self.display_state(self.current_state, "initial state")
                    return True
                else:
                    print("Invalid state! Please ensure you have numbers 0-8 with no duplicates.")
                    
            except Exception as e:
                print(f"Invalid input format! Error: {e}")
                print("Please use the format: [1,2,3,4,0,8,5,6,7]")
    
    def make_move(self, user_input: str) -> bool:
        # process user input and make a move if valid
        key_to_action = {'w': 'up', 's': 'down', 'a': 'left', 'd': 'right'}
        action = key_to_action.get(user_input.lower())
        
        if action:
            new_state = self.apply_action(self.current_state, action)
            if new_state:
                self.current_state = new_state
                self.moves_count += 1
                return True
        
        return False
    
    def play_game(self):
        # main interactive game loop
        self.display_instructions()
        
        if not self.setup_initial_state():
            return
        
        print("\nGoal state:")
        self.display_state(self.goal_state)
        
        while not self.is_goal_state(self.current_state):
            
            move = input("\nEnter your move: ").strip()
            
            if move.lower() == 'q':
                print("Thanks for playing! Goodbye!")
                break
            
            if self.make_move(move):
                clear_output(wait=True)
                print(f"\nMove {self.moves_count} completed!")
                self.display_state(self.current_state, f"Current state (move {self.moves_count})")
                
                if self.is_goal_state(self.current_state):
                    print("\nCongratulations!")
                    print(f"You solved the puzzle in {self.moves_count} moves!")
                    print("Goal state reached!")
                    break
            else:
                print("Invalid move! Please try again.")


#### Run to play the game

In [15]:
game = EightPuzzleGame()
game.play_game()


8-Puzzle Game Start


The 8-puzzle problem is a 3x3 board with 8 tiles numbered from 1 to 8
and one empty space (represented by 0).

The objective is to begin with an arbitrary configuration of tiles,
and move them to place the numbered tiles to match the final configuration.

Rules:
1. Input the initial state of the puzzle using this format:
   [1,2,3,4,0,8,5,6,7] (where 0 represents the empty space)

2. Use the following keys to move tiles into the empty space:
   'W' or 'w' → move tile up into empty space
   'S' or 's' → move tile down into empty space
   'A' or 'a' → move tile left into empty space
   'D' or 'd' → move tile right into empty space
   'Q' or 'q' → quit the game


Enter the initial state of the puzzle:


Format: [1,2,3,4,0,8,5,6,7] →  [1,2,0,4,5,3,7,8,6]



initial state:
┌─────┬─────┬─────┐
│  1  │  2  │     │
├─────┼─────┼─────┤
│  4  │  5  │  3  │
├─────┼─────┼─────┤
│  7  │  8  │  6  │
└─────┴─────┴─────┘

Goal state:

state:
┌─────┬─────┬─────┐
│  1  │  2  │  3  │
├─────┼─────┼─────┤
│  4  │  5  │  6  │
├─────┼─────┼─────┤
│  7  │  8  │     │
└─────┴─────┴─────┘



Enter your move:  w



Move 1 completed!

Current state (move 1):
┌─────┬─────┬─────┐
│  1  │  2  │  3  │
├─────┼─────┼─────┤
│  4  │  5  │     │
├─────┼─────┼─────┤
│  7  │  8  │  6  │
└─────┴─────┴─────┘



Enter your move:  w



Move 2 completed!

Current state (move 2):
┌─────┬─────┬─────┐
│  1  │  2  │  3  │
├─────┼─────┼─────┤
│  4  │  5  │  6  │
├─────┼─────┼─────┤
│  7  │  8  │     │
└─────┴─────┴─────┘

Congratulations!
You solved the puzzle in 2 moves!
Goal state reached!


#### Task 3: Breadth-First Search (BFS) in Python

In [21]:
from collections import deque
from typing import Set, Dict, Any, Optional, Tuple, List
import time

In [27]:
class SearchNode:
    # represents a node in the search tree.
    # generic node class that can be used with any search problem.
    
    def __init__(self, state: Any, parent: Optional['SearchNode'] = None, 
                 action: Optional[str] = None, depth: int = 0, path_cost: int = 0):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = depth
        self.path_cost = path_cost
    
    def __eq__(self, other) -> bool:
        # check if two nodes have the same state
        return self.state == other.state if isinstance(other, SearchNode) else False
    
    def __hash__(self) -> int:
        # make node hashable for use in sets
        return hash(str(self.state))
    
    def get_solution_path(self) -> List[Tuple[Any, str]]:
        # get the path from root to this node
        path = []
        current = self
        while current:
            if current.action:
                path.append((current.state, current.action))
            else:
                path.append((current.state, "initial state"))
            current = current.parent
        return path[::-1]  # reverse to get path from root to current

In [33]:
class BreadthFirstSearch:
    # generic breadth-first search algorithm.
    # can work with any problem class that implements the required interface.
    
    def __init__(self, problem: Any):
        self.problem = problem
        self.statistics = {
            'nodes_explored': 0,
            'nodes_generated': 0,
            'max_frontier_size': 0,
            'time_taken': 0.0,
            'solution_length': -1
        }
    
    def state_to_key(self, state: Any) -> Tuple:
        # convert state to hashable key for duplicate detection
        if isinstance(state, list):
            return tuple(tuple(row) if isinstance(row, list) else row for row in state)
        return tuple(state) if hasattr(state, '__iter__') else state
    
    def search(self, verbose: bool = True) -> Tuple[Optional[SearchNode], Dict[str, Any]]:
        start_time = time.time()
        
        # initialize with root node
        root_node = SearchNode(self.problem.initial_state)
        
        # check if initial state is goal
        if self.problem.is_goal_state(root_node.state):
            self.statistics['time_taken'] = time.time() - start_time
            self.statistics['solution_length'] = 0
            return root_node, self.statistics
        
        # initialize frontier (queue) and explored set
        frontier = deque([root_node])
        explored = set()
        frontier_states = {self.state_to_key(root_node.state)}
        
        self.statistics['nodes_generated'] = 1
        self.statistics['max_frontier_size'] = 1
        
        if verbose:
            print("BFS Initiated")
            print(f"Initial state: {self.problem.initial_state}")
            print(f"Goal state: {self.problem.goal_state}")
            print()
        
        while frontier:
            # update frontier size tracking
            current_frontier_size = len(frontier)
            if current_frontier_size > self.statistics['max_frontier_size']:
                self.statistics['max_frontier_size'] = current_frontier_size
            
            # remove node from frontier (fifo)
            current_node = frontier.popleft()
            current_state_key = self.state_to_key(current_node.state)
            frontier_states.discard(current_state_key)
            
            # add to explored set
            explored.add(current_state_key)
            self.statistics['nodes_explored'] += 1
            
            if verbose and self.statistics['nodes_explored'] % 100 == 0:
                print(f"Nodes explored: {self.statistics['nodes_explored']}, "
                      f"Frontier size: {len(frontier)}")
            
            # expand current node
            valid_actions = self.problem.get_valid_actions(current_node.state)
            
            for action in valid_actions:
                child_state = self.problem.apply_action(current_node.state, action)
                if child_state is None:
                    continue
                
                child_state_key = self.state_to_key(child_state)
                
                # skip if already explored or in frontier
                if child_state_key in explored or child_state_key in frontier_states:
                    continue
                
                # create child node
                child_node = SearchNode(
                    state=child_state,
                    parent=current_node,
                    action=action,
                    depth=current_node.depth + 1,
                    path_cost=current_node.path_cost + 1
                )
                
                self.statistics['nodes_generated'] += 1
                
                # goal test
                if self.problem.is_goal_state(child_state):
                    self.statistics['time_taken'] = time.time() - start_time
                    self.statistics['solution_length'] = child_node.depth
                    
                    if verbose:
                        print("\nsolution found!")
                        print(f"Solution length: {child_node.depth} moves")
                        print(f"Nodes explored: {self.statistics['nodes_explored']}")
                        print(f"Nodes generated: {self.statistics['nodes_generated']}")
                        print(f"Time taken: {self.statistics['time_taken']:.4f} seconds")
                    
                    return child_node, self.statistics
                
                # add to frontier
                frontier.append(child_node)
                frontier_states.add(child_state_key)
        
        # no solution found
        self.statistics['time_taken'] = time.time() - start_time
        
        if verbose:
            print("No solution found!")
            print(f"Nodes explored: {self.statistics['nodes_explored']}")
        
        return None, self.statistics
    
    def print_solution_path(self, solution_node: Optional[SearchNode]):
        # print the complete solution path
        if not solution_node:
            print("No solution to display")
            return
        
        path = solution_node.get_solution_path()
        
        print()
        print("Complete solution path")
        print()
        
        for i, (state, action) in enumerate(path):
            if hasattr(self.problem, 'display_state'):
                self.problem.display_state(state, f"Step {i}: {action}")
            else:
                print(f"Step {i}: {action}")
                print(f"State: {state}")
            
            if i < len(path) - 1:
                print("↓")
        
        print(f"\nTotal moves to reach goal: {len(path) - 1}")

#### Task 3: Breadth-First Search (BFS) for the 8-Puzzle Problem

In [ ]:
import os
from typing import list, optional, dict, any

In [36]:
import time
from typing import List

class EightPuzzleBFS:
    # specialized solver that combines 8-puzzle problem with bfs algorithm.
    # handles file i/o and provides comprehensive solving capabilities.
    
    def __init__(self):
        self.problem = EightPuzzle()
        self.bfs = None
        self.solution = None
        self.statistics = None
    
    def read_from_file(self, filename: str) -> bool:
        # read initial and goal states from file
        try:
            with open(filename, 'r') as file:
                lines = [line.strip() for line in file.readlines() if line.strip()]
            
            if len(lines) < 6:
                raise ValueError("File must contain at least 6 lines")
            
            # parse initial state (first 3 lines)
            initial_state: List[List[int]] = []
            for i in range(3):
                row = [int(x.strip()) for x in lines[i].split(',')]
                if len(row) != 3:
                    raise ValueError(f"Row {i+1} must have exactly 3 numbers")
                initial_state.append(row)
            
            # parse goal state (next 3 lines)
            goal_state: List[List[int]] = []
            for i in range(3, 6):
                row = [int(x.strip()) for x in lines[i].split(',')]
                if len(row) != 3:
                    raise ValueError(f"Row {i-2} must have exactly 3 numbers")
                goal_state.append(row)
            
            # validate states
            if not (self.problem.is_valid_state(initial_state) and 
                    self.problem.is_valid_state(goal_state)):
                raise ValueError("Invalid puzzle states in file.")
            
            self.problem.initial_state = initial_state
            self.problem.goal_state = goal_state
            
            print(f"Successfully loaded puzzle from '{filename}'")
            return True
            
        except FileNotFoundError:
            print(f"Error: file '{filename}' not found!")
            return False
        except Exception as e:
            print(f"Error reading file: {e}")
            return False
    
    
    def solve_puzzle(self, verbose: bool = True) -> bool:
        # solve the loaded puzzle using bfs
        if not self.problem.initial_state or not self.problem.goal_state:
            print("No puzzle loaded! Please load a puzzle first.")
            return False
        
        # check solvability
        if not self.problem.is_solvable():
            print("This puzzle configuration is not solvable.")
            print("The initial and goal states have different inversion parity.")
            return False
        
        if verbose:
            print("\nPuzzle analysis")
            self.problem.display_state(self.problem.initial_state, "Initial state")
            self.problem.display_state(self.problem.goal_state, "Goal state")
            print("Solvable: yes")
        
        # create and run bfs solver
        self.bfs = BreadthFirstSearch(self.problem)
        self.solution, self.statistics = self.bfs.search(verbose=verbose)
        
        return self.solution is not None
    
    def display_solution(self):
        # display the complete solution with detailed output
        if not self.solution:
            print("No solution to display. Run solve_puzzle() first.")
            return
        
        print("\nSolution found!\n")
        
        # print solution path
        self.bfs.print_solution_path(self.solution)
        
        # print detailed statistics
        print("\nDetailed performance analysis\n")
        print("Solution quality:")
        print(f"  • Moves to goal: {self.statistics['solution_length']}")
        
        print("\nSearch efficiency:")
        print(f"  • Nodes explored: {self.statistics['nodes_explored']}")
        print(f"  • Nodes generated: {self.statistics['nodes_generated']}")
        
        print("\nMemory usage:")
        print(f"  • Max frontier size: {self.statistics['max_frontier_size']}")
        
        print("\nTime performance:")
        print(f"  • Execution time: {self.statistics['time_taken']:.4f} seconds")
        
        if self.statistics['nodes_explored'] > 0:
            branching_factor = self.statistics['nodes_generated'] / self.statistics['nodes_explored']
            print("\nSearch tree analysis:")
            print(f"  • Avg branching factor: {branching_factor:.2f}")
            print(f"  • Tree depth: {self.statistics['solution_length']}")
    
    def run_interactive_mode(self):
        # interactive solver with menu system
        print("8-Puzzle BFS solver")
        print("Combining modular 8-puzzle problem with generic BFS algorithm.")
        
        try:
            while True:
                print("\nMenu options:")
                print("1. Load puzzle from file")
                print("2. Manual puzzle input") 
                print("3. Solve current puzzle")
                print("4. View solution details")
                print("5. Exit")
                
                choice = input("\nEnter choice (1-5): ").strip()
                
                if choice == '1':
                    filename = input("Enter filename: ").strip()
                    self.read_from_file(filename)
                
                elif choice == '2':
                    self.manual_input()
                
                elif choice == '3':
                    if self.solve_puzzle():
                        print("\nPuzzle solved successfully.")
                        print("Use option 4 to view detailed solution.")
                    else:
                        print("\nCould not solve puzzle.")
                
                elif choice == '4':
                    self.display_solution()
                
                elif choice == '5':
                    print("Thank you for using the 8-puzzle solver. Goodbye.")
                    break
                
                else:
                    print("Invalid choice. Please enter a number from 1 to 5.")
        except KeyboardInterrupt:
            print("\nInteractive session cancelled by user.")
    
    def manual_input(self):
        # manual input interface for puzzle states
        print("\nManual puzzle input")
        print("Enter each row as comma-separated values (e.g., 1,2,3). Use 0 to represent the empty space.")
        
        try:
            # input initial state
            print("\nInitial state:")
            initial_state: List[List[int]] = []
            for i in range(3):
                while True:
                    try:
                        row_input = input(f"  row {i+1}: ").strip()
                        row = [int(x.strip()) for x in row_input.split(',')]
                        if len(row) != 3:
                            print("  Please enter exactly 3 numbers.")
                            continue
                        initial_state.append(row)
                        break
                    except ValueError:
                        print("  Please enter valid integers separated by commas.")
            
            # input goal state  
            print("\nGoal state:")
            goal_state: List[List[int]] = []
            for i in range(3):
                while True:
                    try:
                        row_input = input(f"  row {i+1}: ").strip()
                        row = [int(x.strip()) for x in row_input.split(',')]
                        if len(row) != 3:
                            print("  Please enter exactly 3 numbers.")
                            continue
                        goal_state.append(row)
                        break
                    except ValueError:
                        print("  Please enter valid integers separated by commas.")
            
            # validate states
            if not (self.problem.is_valid_state(initial_state) and 
                    self.problem.is_valid_state(goal_state)):
                print("Invalid puzzle states. Each must contain numbers 0-8 exactly once.")
                return
            
            self.problem.initial_state = initial_state
            self.problem.goal_state = goal_state
            
            print("\nPuzzle states accepted.")
            self.problem.display_state(initial_state, "Entered initial state")
            self.problem.display_state(goal_state, "Entered goal state")
            
        except KeyboardInterrupt:
            print("\nInput cancelled.")
    

In [37]:
solve = EightPuzzleBFS()
solve.run_interactive_mode()

8-Puzzle BFS solver
Combining modular 8-puzzle problem with generic BFS algorithm.

Menu options:
1. Load puzzle from file
2. Manual puzzle input
3. Solve current puzzle
4. View solution details
5. Exit



Enter choice (1-5):  1
Enter filename:  easy_puzzle.txt


Successfully loaded puzzle from 'easy_puzzle.txt'

Menu options:
1. Load puzzle from file
2. Manual puzzle input
3. Solve current puzzle
4. View solution details
5. Exit



Enter choice (1-5):  3



Puzzle analysis

Initial state:
┌─────┬─────┬─────┐
│  1  │  2  │  3  │
├─────┼─────┼─────┤
│  4  │     │  6  │
├─────┼─────┼─────┤
│  7  │  5  │  8  │
└─────┴─────┴─────┘

Goal state:
┌─────┬─────┬─────┐
│  1  │  2  │  3  │
├─────┼─────┼─────┤
│  4  │  5  │  6  │
├─────┼─────┼─────┤
│  7  │  8  │     │
└─────┴─────┴─────┘
Solvable: yes
BFS Initiated
Initial state: [[1, 2, 3], [4, 0, 6], [7, 5, 8]]
Goal state: [[1, 2, 3], [4, 5, 6], [7, 8, 0]]


solution found!
Solution length: 2 moves
Nodes explored: 2
Nodes generated: 6
Time taken: 0.0001 seconds

Puzzle solved successfully.
Use option 4 to view detailed solution.

Menu options:
1. Load puzzle from file
2. Manual puzzle input
3. Solve current puzzle
4. View solution details
5. Exit


...

Successfully loaded puzzle from 'hard_puzzle.txt'

Menu options:
1. Load puzzle from file
2. Manual puzzle input
3. Solve current puzzle
4. View solution details
5. Exit



Enter choice (1-5):  3



Puzzle analysis

Initial state:
┌─────┬─────┬─────┐
│  8  │  6  │  7  │
├─────┼─────┼─────┤
│  2  │  5  │  4  │
├─────┼─────┼─────┤
│  3  │     │  1  │
└─────┴─────┴─────┘

Goal state:
┌─────┬─────┬─────┐
│  1  │  2  │  3  │
├─────┼─────┼─────┤
│  4  │  5  │  6  │
├─────┼─────┼─────┤
│  7  │  8  │     │
└─────┴─────┴─────┘
Solvable: yes
BFS Initiated
Initial state: [[8, 6, 7], [2, 5, 4], [3, 0, 1]]
Goal state: [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

Nodes explored: 100, Frontier size: 75
Nodes explored: 200, Frontier size: 121
Nodes explored: 300, Frontier size: 203
Nodes explored: 400, Frontier size: 255
Nodes explored: 500, Frontier size: 282
Nodes explored: 600, Frontier size: 369
Nodes explored: 700, Frontier size: 458
Nodes explored: 800, Frontier size: 540
Nodes explored: 900, Frontier size: 566
Nodes explored: 1000, Frontier size: 610
Nodes explored: 1100, Frontier size: 648
Nodes explored: 1200, Frontier size: 683
Nodes explored: 1300, Frontier size: 729
Nodes explored: 1400, Front


Enter choice (1-5):  5


Thank you for using the 8-puzzle solver. Goodbye.
